# Composable AI systems: Building and AI Stylist Specialist selling our products


In [0]:
%pip install databricks-sdk==0.41.0 langchain-community==0.2.10 langchain-openai==0.1.19 mlflow==2.20.2 faker==33.1.0
dbutils.library.restartPython()

In [0]:
%run ./_resources/00-init-stylist $reset_all=false

## Creating our tools: Using Unity Catalog Functions

Let's start by defining the functions our LLM will be able to execute. These functions can contain any logic, from simple SQL to advanced python.

### Computing Mathematics: converting inches to centimeters

Our online shop is selling across all regions, and we know our customers often ask to convert cm to inches. However, plain LLMs are quite bad at executing math. To solve this, let's create a simple function doing the conversion.

We'll save this function within Unity Catalog. You can open the explorer to review the functions created in this notebook.

*Note: This is a very simple first example for this demo. We'll implement a broader math tool later on.*

In [0]:
%sql
CREATE OR REPLACE FUNCTION convert_inch_to_cm(size_in_inch FLOAT)
RETURNS FLOAT
LANGUAGE SQL
COMMENT 'convert size from inch to cm'
RETURN size_in_inch * 2.54;

-- let's test our function:
SELECT convert_inch_to_cm(10) as 10_inches_in_cm;

### Executing a tool to fetch internal data: getting the latest customer orders

We want our stylist assistant to be able to list all existing customer orders

In [0]:
%sql
CREATE OR REPLACE FUNCTION get_customer_orders ()
RETURNS TABLE(user_id STRING,
  id STRING,
  transaction_date STRING,
  item_count DOUBLE,
  amount DOUBLE,
  order_status STRING)
COMMENT 'Returns a list of customer orders for the given customer ID (expect a UUID)'
LANGUAGE SQL
    RETURN
    SELECT o.* from tools_orders o 
    inner join tools_customers c on c.id = o.user_id 
    where email=current_user() ORDER BY transaction_date desc;

SELECT * FROM get_customer_orders();

### Executing a Python function to fetch external dataset in realtime: getting the weather

We want our stylist assistant to give us recommendations based on the weather. Let's add a tool to fetch the weather based on longitude/latitude, using Python to call an external Weather API.

**Note: This will be run by a serverless compute, and accessing external data, therefore requires serverless network egress access. If this fails in a serverless setup/playground, make sure you are allowing it in your networking configuration (open the workspace network option at admin account level)**

In [0]:
%sql
CREATE OR REPLACE FUNCTION get_weather(latitude DOUBLE, longitude DOUBLE)
RETURNS STRUCT<temperature_in_celsius DOUBLE, rain_in_mm DOUBLE>
LANGUAGE PYTHON
COMMENT 'This function retrieves the current temperature and rain information for a given latitude and longitude using the Open-Meteo API.'
AS
$$
  try:
    import requests as r
    #Note: this is provided for education only, non commercial - please get a license for real usage: https://api.open-meteo.com. Let s comment it to avoid issues for now
    #weather = r.get(f'https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,rain&forecast_days=1').json()
    return {
      "temperature_in_celsius": weather["current"]["temperature_2m"],
      "rain_in_mm": weather["current"]["rain"]
    }
  except:
    return {"temperature_in_celsius": 25.0, "rain_in_mm": 0.0}
$$;

-- let's test our function:
SELECT get_weather(52.52, 13.41) as weather;

### Creating a function calling LLMs with specific prompt as a tool

You can also register tools containing custom prompts that your LLM can use to to execute actions based on the customer context.

Let's create a tool that recommend the style for our user, based on the current weather.

In [0]:
%sql
CREATE OR REPLACE FUNCTION recommend_outfit_description(requested_style STRING, temperature_in_celsius FLOAT, rain_in_mm FLOAT)
RETURNS STRING
LANGUAGE SQL
COMMENT 'This function generate a stylist outfit description based on initial request and the current weather.'
RETURN SELECT ai_query("databricks-meta-llama-3-3-70b-instruct",
    CONCAT("You are a stylist assistant. Your goal is to give recommendation on what would be the best outfit for today. The current temperature is ",temperature_in_celsius ," celsius and rain is:", rain_in_mm, "mm. Give size in inches if any. Don't assume customer size if they don't share it. Give ideas of colors and other items to match. The user added this instruction based on what they like: ", requested_style)
  ) AS recommended_outfit;

-- let's test our function:
SELECT recommend_outfit_description("I need a dress for an interview.", 30.1, 0.0)

### Using Vector search to find similar content as a tool

Let's now add a tool to recomment similar items, based on an article description.

We'll be using Databricks Vector Search to perform a realtime similarity search and return articles that we could suggest from our database.

To do so, you can leverage the new `vector_search` SQL function. See [Documentation](https://docs.databricks.com/en/sql/language-manual/functions/vector_search.html) for more details.

<div style="background-color: #e3efff"> To simplify this demo, we'll fake the call to the vector_search and make it work without pre-loading the demo</div>

In [0]:
%sql
-- Commenting this as the index isn't pre-loaded in the demo to simplify the experience.
-- CREATE OR REPLACE FUNCTION find_clothes_matching_description (description STRING)
-- RETURNS TABLE (category STRING, color STRING, id STRING, price DOUBLE, search_score DOUBLE)
-- LANGUAGE SQL
-- COMMENT 'Finds clothes in our catalog matching the description using AI query and returns a table of results.'
-- RETURN
--  SELECT * FROM VECTOR_SEARCH(index => 'catalog.schema.clothing_index', query => query, num_results => 3) vs


In [0]:
%sql
-- Generate fake data instead of calling the VS index with vector_search;
CREATE OR REPLACE FUNCTION find_clothes_matching_description (description STRING)
RETURNS TABLE (id BIGINT, name STRING, color STRING, category STRING, price DOUBLE, description STRING)
COMMENT 'Finds existing clothes in our catalog matching the description using AI query and returns a table of results.'
RETURN
SELECT clothes.* FROM (
  SELECT explode(from_json(
    ai_query(
      'databricks-meta-llama-3-3-70b-instruct', 
      CONCAT(
        'returns a json list of 3 json object clothes: <"id": bigint, "name": string, "color": string, "category": string, "price": double, "description": string>. These clothes should match the following user description: ', description, '. Return only the answer as a javascript json object ready to be parsed, no comment or text or javascript or ``` at the beginning.' ) ), 'ARRAY<STRUCT<id: BIGINT, name: STRING, color: STRING, category: STRING, price: DOUBLE, description: STRING>>' )) AS clothes );

-- let's test our function:
SELECT * FROM find_clothes_matching_description('a red dress');

## Using Databricks Playground to test our functions

Databricks Playground provides a built-in integration with your functions. It'll analyze which functions are available, and call them to properly answer your question.


<img src="https://github.com/databricks-demos/dbdemos-resources/blob/main/images/product/llm-tools-functions/llm-tools-functions-playground.gif?raw=true" style="float: right; margin-left: 10px; margin-bottom: 10px;">

To try out our functions with playground:
- Open the [Playground](/ml/playground) 
- Select a model supporting tools (like Llama3.1)
- Add the functions you want your model to leverage (`catalog.schema.function_name`)
- Ask a question (for example to convert inch to cm), and playground will do the magic for you!

<br/>
<div style="background-color: #d4e7ff; padding: 10px; border-radius: 15px;clear:both">
<strong>Note:</strong> Tools in playground is in preview, reach-out your Databricks Account team for more details and to enable it.
</div>



## Building an AI system leveraging our Databricks UC functions with Langchain

These tools can also directly be leveraged on custom model. In this case, you'll be in charge of chaining and calling the functions yourself (the playground does it for you!)

Langchain makes it easy for you. You can create your own custom AI System using a Langchain model and a list of existing tools (in our case, the tools will be the functions we just created)

### Enable MLflow Tracing

Enabling MLflow Tracing is required to:
- View the chain's trace visualization in this notebook
- Capture the chain's trace in production via Inference Tables
- Evaluate the chain via the Mosaic AI Evaluation Suite

In [0]:
import mlflow
mlflow.langchain.autolog()

### Start by creating our tools from Unity Catalog

Let's use UCFunctionToolkit to select which functions we want to use as tool for our demo:

In [0]:
from langchain_community.tools.databricks import UCFunctionToolkit
import pandas as pd
wh = get_shared_warehouse(name = None) #Get the first shared wh we can. See _resources/01-init for details
print(f'This demo will be using the wg {wh.name} to execute the functions')

def get_tools():
    return (
        UCFunctionToolkit(warehouse_id=wh.id)
        # Include functions as tools using their qualified names.
        # You can use "{catalog_name}.{schema_name}.*" to get all functions in a schema.
        .include(f"{catalog}.{db}.*")
        .get_tools())

display_tools(get_tools()) #display in a table the tools - see _resource/00-init for details

### Let's create our langchain agent using the tools we just created

In [0]:

from langchain_openai import ChatOpenAI
from databricks.sdk import WorkspaceClient

# Note: langchain_community.chat_models.ChatDatabricks doesn't support create_tool_calling_agent yet - it'll soon be availableK. Let's use ChatOpenAI for now
llm = ChatOpenAI(
  base_url=f"{WorkspaceClient().config.host}/serving-endpoints/",
  api_key=dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get(),
  model="databricks-meta-llama-3-3-70b-instruct"
)


In [0]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.chat_models import ChatDatabricks

def get_prompt(history = [], prompt = None):
    if not prompt:
            prompt = """You are a helpful fashion assistant. Your task is to recommend cothes to the users. You can use the following tools:
    - Use the convert_inch_to_cm to covert inch to cm if the customer ask for it
    - Use get_customer_orders to get the list of orders and their status, total amount and number of article. 
    - Use find_clothes_matching_description to find existing clothes from our internal catalog and suggest article to the customer
    - Use recommend_outfit if a customer ask you for a style. This function take the weather as parameter. Use the get_weather function first before calling this function to get the current temperature and rain. The current user location is 52.52, 13.41.

    Make sure to use the appropriate tool for each step and provide a coherent response to the user. Don't mention tools to your users. Only answer what the user is asking for. If the question isn't related to the tools or style/clothe, say you're sorry but can't answer"""
    return ChatPromptTemplate.from_messages([
            ("system", prompt),
            ("human", "{input}"),
            ("placeholder", "{agent_scratchpad}"),
    ])

In [0]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
prompt = get_prompt()
tools = get_tools()
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

### Let's give it a try: Asking to run a simple size conversion.
Under the hood, we want this to call our Math conversion function:

In [0]:
#make sure our log is enabled to properly display and trace the call chain 
mlflow.langchain.autolog()
agent_executor.invoke({"input": "what's 12in in cm?"})

In [0]:
agent_executor.invoke({"input": "what are my latest orders?"})

### Chaining multiple tools

Let's see if the agent can properly chain the calls, using multiple tools and passing the values from one to another. We'll ask for an outfit for today, and the expectation is that the LLM will fetch the weather before.

In [0]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
answer = agent_executor.invoke({"input": "I need a dress for an interview I have today. What style would you recommend for today?"})

In [0]:
#Note: in a real app, we would include the preview discussion as history to keep the reference.
answer = agent_executor.invoke({"input": "Can you give me a list of red dresses I can buy?"})

In [0]:
displayHTML(answer['output'].replace('\n', '<br>'))

## Extra: running more advanced functions using python:

Let's see a few more advanced tools example

### Calculator tool: Supporting math operations 
Let's add a function to allow our LLM to execute any Math operation. 

Databricks runs the python in a safe container. However, we'll filter what the function can do to avoid any potential issues with prompt injection (so that the user cannot execute other python instructions).

In [0]:
%sql
CREATE OR REPLACE FUNCTION compute_math(expr STRING)
RETURNS STRING
LANGUAGE PYTHON
COMMENT 'Run any mathematical function and returns the result as output. Supports python syntax like math.sqrt(13)'
AS
$$
  import ast
  import operator
  import math
  operators = {ast.Add: operator.add, ast.Sub: operator.sub, ast.Mult: operator.mul, ast.Div: operator.truediv, ast.Pow: operator.pow, ast.Mod: operator.mod, ast.FloorDiv: operator.floordiv, ast.UAdd: operator.pos, ast.USub: operator.neg}
    
  # Supported functions from the math module
  functions = {name: getattr(math, name) for name in dir(math) if callable(getattr(math, name))}

  def eval_node(node):
    if isinstance(node, ast.Num):  # <number>
      return node.n
    elif isinstance(node, ast.BinOp):  # <left> <operator> <right>
      return operators[type(node.op)](eval_node(node.left), eval_node(node.right))
    elif isinstance(node, ast.UnaryOp):  # <operator> <operand> e.g., -1
      return operators[type(node.op)](eval_node(node.operand))
    elif isinstance(node, ast.Call):  # <func>(<args>)
      func = node.func.id
      if func in functions:
        args = [eval_node(arg) for arg in node.args]
        return functions[func](*args)
      else:
        raise TypeError(f"Unsupported function: {func}")
    else:
      raise TypeError(f"Unsupported type: {type(node)}")  
  try:
    if expr.startswith('```') and expr.endswith('```'):
      expr = expr[3:-3].strip()      
    node = ast.parse(expr, mode='eval').body
    return eval_node(node)
  except Exception as ex:
    return str(ex)
$$;
-- let's test our function:
SELECT compute_math("(2+2)/3") as result;

### Run any python function

If we are creating a coding assistant, it can be useful to allow our AI System to run python code to try them and output the results to the user. **Be careful doing that, as any code could be executed by the user.**

Here is an example:

In [0]:
%sql
CREATE OR REPLACE FUNCTION execute_python_code(python_code STRING)
RETURNS STRING
LANGUAGE PYTHON
COMMENT "Run python code. The code should end with a return statement and this function will return it as a string. Only send valid python to this function. Here is an exampe of python code input: 'def square_function(number):\\n  return number*number\\n\\nreturn square_function(3)'"
AS
$$
    import traceback
    try:
        import re
        # Remove code block markers (e.g., ```python) and strip whitespace```
        python_code = re.sub(r"^\s*```(?:python)?|```\s*$", "", python_code).strip()
        # Unescape any escaped newline characters
        python_code = python_code.replace("\\n", "\n")
        # Properly indent the code for wrapping
        indented_code = "\n    ".join(python_code.split("\n"))
        # Define a wrapper function to execute the code
        exec_globals = {}
        exec_locals = {}
        wrapper_code = "def _temp_function():\n    "+indented_code
        exec(wrapper_code, exec_globals, exec_locals)
        # Execute the wrapped function and return its output
        result = exec_locals["_temp_function"]()
        return result
    except Exception as ex:
        return traceback.format_exc()
$$;
-- let's test our function:

SELECT execute_python_code("return 'Hello World! '* 3") as result;

In [0]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
prompt = get_prompt(prompt="You are an assistant for a python developer. Internally, you have a tool named execute_python_code that can generate and run python code to help answering what the customer is asking. input: valid python code as a string. output: the result of the return value from the code execution. Don't mention you have tools or the tools name. Make sure you send the full python code at once to the function and that the code has a return statement at the end to capture the result. Don't print anything in the code you write, return the result you need as final instruction. Make sure the python code is valid. Only send python. Here is an example: 'def square_function(number):\\n  return number*number\\n\\nreturn square_function(3)'")
tools = get_tools()
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

#Let's print the answer! Note that the LLM often makes a few error, but then analyze it and self-correct.
answer = agent_executor.invoke({"input": "What's the result of the fibonacci suite? Display its result for 5."})
displayHTML(answer['output'].replace('\n', '<br>'))

## Deploying our Agent Executor as Model Serving Endpoint

We're now ready to package our chain within MLFLow, and deploy it as a Model Serving Endpoint, leveraging Databricks Agent Framework and its review application.

In [0]:
from langchain.schema.runnable import RunnableLambda
from langchain_core.output_parsers import StrOutputParser

# Function to extract the user's query
def extract_user_query_string(chat_messages_array):
    return chat_messages_array[-1]["content"]

# Wrapping the agent_executor invocation
def agent_executor_wrapper(input_data):
    result = agent_executor.invoke({"input": input_data})
    return result["output"]

# Create the chain using the | operator with StrOutputParser
chain = (
    RunnableLambda(lambda data: extract_user_query_string(data["messages"]))  # Extract the user query
    | RunnableLambda(agent_executor_wrapper)  # Pass the query to the agent executor
    | StrOutputParser()  # Optionally parse the output to ensure it's a clean string
)

In [0]:
# Example input data
input_data = {
    "messages": [
        {"content": "Write a function that computes the Fibonacci sequence in Python and displays its result for 5."}
    ]
}
# Run the chain
answer = chain.invoke(input_data)
displayHTML(answer.replace('\n', '<br>'))

## Deploying the agent as a model serving endpoint

Databricks automatically generates all the required notebooks and setup for you to deploy these agents!

To deploy them with the latest configuration, open [Databricks Playground ](/ml/playground), select the tools you want to use and click on the "Export Notebook" button on the top!

This will generate notebooks pre-configured for you, including the review application to start testing and collecting your new model!

## Conclusion
That's it, we saw how you can leverage Databricks Functions as tools within your AI system.

This demo gave you a simple overview and some examples to get started. You can take it to the next level within your own AI Systems!

Once your application is complete and your chain ready to be deployed, you can easily serve your model as a Model Serving Endpoint, acting as your Compound AI System!

### Coming soon

We'll soon update this demo to add more examples, including:
- how to properly leverage secrets within AI functions to call external systems having Authentication
- generating images and calling ML model endpoints
- More to come!

Stay tuned!